In [ ]:
import ee
# Import the folium library.
import folium
from folium import plugins
import logging
## Trigger the authentication flow. You only need to do this once
## To be able to access google earth engine you have to register first
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=uYUzXtb6NOu5SzuZ_99bXb70tUW_J6u-TbzxRGDwpv8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5w1--rgvpdKm5zS9_WXLft_U6My9h1tE1pHNpShLxKNt76EAFMSA0

Successfully saved authorization token.


In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
# code adapted from here: https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=ul0CTLBS5kcx

def add_ee_layer(self, dataset, opacity= 0.7):
    ee_object = dataset.ee_object
    vis_params = dataset.vis_params
    name= dataset.name 
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
#            print(map_id_dict['tile_fetcher'].url_format)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True,
            opacity= opacity 
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            print(map_id_dict['tile_fetcher'].url_format)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True,
            opacity= opacity
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
       ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
#            print(map_id_dict['tile_fetcher'])
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except Exception as e:
      print(e)

    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:

# Possible basemaps
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Define layers as you want from earth engine datasets here:  https://developers.google.com/earth-engine/datasets

class NVDI:
  #https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MYD13Q1
  name = 'Aqua Vegetation Indices 16-Day Global 250m'
  ee_object = ee.ImageCollection('MODIS/006/MYD13Q1').filter(ee.Filter.date('2016-01-01', '2017-05-01')).select('NDVI')
  vis_params = {
    'min': 0.0,
    'max': 8000.0,
    'palette': [
      'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
      '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
      '012E01', '011D01', '011301'
    ]
  }

#https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF
# data from 2007 to 2017:
class fnf:
  name = 'Global PALSAR-2/PALSAR Forest/Non-Forest Map -2017'
  ee_object = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF').filterDate('2007-01-01', '2007-12-31').select('fnf')
  vis_params = {
    'min': 1.0,
    'max': 3.0,
    'palette': ['006400', 'FEFF99', '0000FF'],
  }

class CGLC:
  #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#bands
  # Any year from 2015-2019, looking in 'tree-coverfraction' band (many others)
  name = 'Copernicus Global Land Cover Layers: CGLS-LC100 collection 3 - Discrete Classification - 2019'
  ee_object = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification');
  vis_params = {}

class planet:
  #https://developers.google.com/earth-engine/datasets/catalog/SKYSAT_GEN-A_PUBLIC_ORTHO_RGB#bands
  name = "Planet SkySat Public Ortho Imagery, RGB"

  ee_object = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB').select(['R', 'G', 'B'])
  vis_params = {
    'bands': ['R', 'G', 'B'],
    'min': 11.0,
    'max': 190.0
  }


class landsat8:
#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR#description
  name = "USGS Landsat 8 Surface Reflectance Tier 1"

  def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

  ee_object = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2019-01-01', '2019-12-31').filter(ee.Filter.lt('CLOUD_COVER',5)).map(maskL8sr)
  vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
  }

class sentinel2:
  #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR?hl=en
  name = 'Sentinel-2 MSI: MultiSpectral Instrument, Level-2A'

  def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

  ee_object = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2019-01-01', '2019-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',1)).map(maskS2clouds)
  vis_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
  }



In [ ]:
# Base configuration of folium maps

# Create a folium map object (centred on DSTI :)).
my_map = folium.Map(location=[43.61529927557995, 7.075464799049402], zoom_start=12, height=500)

# you can hide or unhide the layers in the left icon or by uncommenting bellow
basemaps['Esri Satellite'].add_to(my_map)
my_map.add_ee_layer(NVDI, opacity=0.8)
my_map.add_ee_layer(fnf, opacity=0.7)
#my_map.add_ee_layer(CGLC, opacity=0.7)
#my_map.add_ee_layer(planet, opacity=0.7)
#my_map.add_ee_layer(landsat8, opacity=0.9)
my_map.add_ee_layer(sentinel2, opacity=0.9)

# Add a layer control panel and fullscreen button to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/43455307d7afaf7275bce671ae070691-af9cc4c8ce32cc61df66c9f3697bb449/tiles/{z}/{x}/{y}
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/b1043a24477c7d956b5e9d64e5033e0f-ae3adcc22202fe2aa341bb7d764b3d32/tiles/{z}/{x}/{y}
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/eb489f4d0f32f131771a2e5fd9916493-4100337a974d10100ebb3f580ac6b5cd/tiles/{z}/{x}/{y}
